In [1]:
## Data collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

##load the dataset
df = gutenberg.raw('shakespeare-hamlet.txt')

## save to file
with open('hamlet.txt','w') as file:
    file.write(df)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Waleed\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load data set
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

## Tokenize the text- creating indexes for words
tokenizer =  Tokenizer()
tokenizer.fit_on_texts([text])
total_words =  len(tokenizer.word_index)+1
total_words




4818

In [3]:
## Creating Input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [15]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [16]:
#pad sequence
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len
 

14

In [17]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [18]:
## Create predictors and label
import tensorflow as tf
X,y= input_sequences[:,:-1],input_sequences[:,-1]


In [19]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [20]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [21]:
y= tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
### Train test split the data set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [23]:
#Train our lstm rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense,Dropout

# #Define model
# model = Sequential()
# model.add(Embedding(total_words,100,input_length=max_sequence_len))

In [26]:
# Define the input and embedding dimensions
input_dim = total_words   # Vocabulary size
output_dim = 100          # Embedding output size
input_length = max_sequence_len  # Sequence length

# Create the Sequential model
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length - 1))
model.add(GRU(150, return_sequences=True))  # Return full sequence from the first GRU layer
model.add(Dropout(0.2))  # Dropout to prevent overfitting
model.add(GRU(100))      # Second GRU layer processes the full sequence
model.add(Dense(total_words, activation="softmax"))  # Final dense layer with softmax activation

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

# Summary to check the model's architecture and parameters
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
## Train the model
history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),verbose=-1)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [28]:
 ##function to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list =  tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] #ensure sequence length matches max_sequre
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted= model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index== predicted_word_index:
            return word
    return None

In [32]:
input_text = "I thinke I heare them. Stand: who's"
print(f"Input Text:{input_text}")
max_sequence_len =  model.input_shape[1]+1
next_word= predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word Prediction:{next_word}")



Input Text:I thinke I heare them. Stand: who's
Next word Prediction:there


In [33]:
## save the model
model.save("next_word_GRU.h5")
## save the tokenizer
import pickle
with open ('tokenizer.pickle','wb') as file:
    pickle.dump(tokenizer,file,protocol=pickle.HIGHEST_PROTOCOL)